## Reference :

- https://towardsdatascience.com/t-sne-clearly-explained-d84c537f53a
- https://www.youtube.com/watch?v=MnRskV3NY1k&t=1571s&ab_channel=T%C3%BCbingenMachineLearning
- https://www.dailydoseofds.com/formulating-and-implementing-the-t-sne-algorithm-from-scratch/

# 2. (5 pts) Dimension Reduction Using t-SNE

## 2.1 (2 pts) Please build the t-SNE algorithm from scratch based on the equations below:

Please cite if you are referring to any source for the algorithms.
There are many hyperparameters to optimize, such as initialization (random seed), learning rate 𝜆 , momentum
𝛼(𝑡), iteration number, and perplexity.



$$

P_{j|i} = \frac{\exp\left(- \frac{\|x_i - x_j\|^2}{2\sigma_i^2}\right)}{\sum_{k \neq i} \exp\left(- \frac{\|x_i - x_k\|^2}{2\sigma_i^2}\right)}

\\ 

\[
q_{j|i} = \frac{(1 + \|y_i - y_j\|^2)^{-1}}{\sum_{k \neq l} (1 + \|y_k - y_l\|^2)^{-1}}
\]

\\ 

\[
C = KL(P||Q) = \sum_i \sum_j P_{j|i} \log \frac{P_{j|i}}{q_{j|i}}
\]

\\

\[
\frac{\partial C}{\partial y_i} = 4 \sum_j (p_{ij} - q_{ij})(y_i - y_j)(1 + \|y_i - y_j\|^2)^{-1}
\]

\\

\[
y^{(t)} = y^{(t-1)} + \lambda \frac{\partial C}{\partial y} + \alpha(t)(y^{(t-1)} - y^{(t-2)})
\]

$$

In [13]:
import pandas as pd

df = pd.read_csv('fashion_mnist1.csv')
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values
print(f"{X.shape = }, {y.shape = }")

X.shape = (10000, 784), y.shape = (10000,)


In [ ]:
import sys
import numpy as np
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")


class TSNE:
    
    def __init__(self, perplexity=10, iterations=1000, early_exaggeration=4,n_components=2, learning_rate=100, momentum=0.5, initialization_method ='random', random_state=42):
        """
        t-SNE algorithm parameters
            :param perplexity: Perplexity
            :param iterations: number of Iterations the gradient descent algorithm will run
            :param early_exaggeration: Early Exaggeration
            :param n_components: Number of components
            :param learning_rate: Learning Rate
            :param momentum: momentum
            :param initialization_method: Initialization method for the t-SNE
            :param random_state: Random State
        """
        self.perplexity = perplexity
        self.iterations = iterations
        self.early_exaggeration = early_exaggeration
        self.n_components = n_components
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.initialization_method = initialization_method
        self.random_state = random_state
    
    def _find_sigma(self, norm, i, perplexity):
        """
        Helper function to obtain σ's based on user-specified perplexity.
            :param norm: pairwise squared differences between data points
            :param i: Iteration number
            :param perplexity: desired Perplexity
        
        return: sigma that satisfies the perplexity condition.
        """
        result = np.inf  # Set first result to be infinity
        std_norm = np.std(norm)  # Use standard deviation of norms to define search space
        sigma = 0
        for sigma_i in np.linspace(0.01 * std_norm, 5 * std_norm, 200):
            # Equation 1 Numerator
            p = np.exp(-(norm**2) / (2 * sigma_i**2))
            p[i] = 0
    
            p_new = np.maximum(p / np.sum(p), sys.float_info.min)
    
            # Shannon Entropy
            H = -np.sum(p_new * np.log2(p_new))
    
            # Get log(perplexity equation) as close to equality
            if np.abs(np.log(perplexity) - H * np.log(2)) < np.abs(result):
                result = np.log(perplexity) - H * np.log(2)
                sigma = sigma_i
    
        return sigma
    
    def get_original_pairwise_affinities(self, X, perplexity):
        """
        param:
            X - input data
            perplexity: perplexity
    
        return: original high dimension probability
        """
    
        n = X.shape[0]
        print("Starting the high dimension computation...")
        p_ij = np.zeros(shape=(n, n))
        with tqdm(total=n) as _:
            for i in range(0, n):
                diff = X[i] - X
                
                norm = np.linalg.norm(diff, axis=1)
                sigma_i = self._find_sigma(norm, i, perplexity)  
                p_ij[i, :] = np.exp(-(norm**2) / (2 * sigma_i**2))
                np.fill_diagonal(p_ij, 0)
                p_ij[i, :] = p_ij[i, :] / np.sum(p_ij[i, :])
    
        p_ij = np.maximum(p_ij, sys.float_info.min)
        print("Completed the high dimension computation...")
        return p_ij
    
    def get_symmetric_p_ij(self, p_ij):
        """
        Function to obtain symmetric affinities matrix utilized in t-SNE.
    
        Parameters:
        p_ij (np.ndarray): The input affinity matrix.
    
        Returns:
        np.ndarray: The symmetric affinities matrix.
    
        """
        # print("Computing Symmetric p_ij matrix....")
        # 
        # n = len(p_ij)
        # p_ij_symmetric = np.zeros(shape=(n, n))
        # for i in range(0, n):
        #     for j in range(0, n):
        #         p_ij_symmetric[i, j] = (p_ij[i, j] + p_ij[j, i]) / (2 * n)
        # 
        # # Set 0 values to minimum numpy value (ε approx. = 0)
        # ε = np.nextafter(0, 1)
        # p_ij_symmetric = np.maximum(p_ij_symmetric, ε)
        # 
        # print("Completed Symmetric p_ij Matrix. \n")
        # 
        # return p_ij_symmetric
        n = p_ij.shape[0]
        p_ij = (p_ij + p_ij.T) / (2 * n)
        p_ij = np.maximum(p_ij, sys.float_info.min)
        return p_ij
    
    def get_low_dimensional_affinities(self, Y):
        """
        Obtain low-dimensional affinities.
    
        Parameters:
        Y (np.ndarray): The low-dimensional representation of the data points.
    
        Returns:
        np.ndarray: The low-dimensional affinities matrix.
        """
    
        n = len(Y)
        q_ij = np.zeros(shape=(n, n))
    
        for i in range(0, n):
            # Equation 4 Numerator
            diff = Y[i] - Y
            norm = np.linalg.norm(diff, axis=1)
            q_ij[i, :] = (1 + norm**2) ** (-1)
    
        # Set p = 0 when j = i
        np.fill_diagonal(q_ij, 0)
    
        # Equation 4
        q_ij = q_ij / q_ij.sum()
    
        # Set 0 values to minimum numpy value (ε approx. = 0)
        q_ij = np.maximum(q_ij, sys.float_info.min)
    
        return q_ij
    
    def _gradient(self, p_ij, q_ij, Y):
        """
        Obtain gradient of cost function at current point Y.
    
        Parameters:
        p_ij (np.ndarray): The joint probability distribution matrix.
        q_ij (np.ndarray): The Student's t-distribution matrix.
        Y (np.ndarray): The current point in the low-dimensional space.
    
        Returns:
        np.ndarray: The gradient of the cost function at the current point Y.
        """
    
        n = len(p_ij)
    
        # Compute gradient
        gradient = np.zeros(shape=(n, Y.shape[1]))
        for i in range(0, n):
            # Equation 5
            diff = Y[i] - Y
            A = np.array([(p_ij[i, :] - q_ij[i, :])])
            B = np.array([(1 + np.linalg.norm(diff, axis=1)) ** (-1)])
            C = diff
            gradient[i] = 4 * np.sum((A * B).T * C, axis=0)
    
        return gradient
    
    def initialization(self, X, initialization_method) :
        """
        Initial t-SNE either randomly or using PCA.
            :param X: input data
            :param initialization_method: initialization method can be random or PCA
        
        :return - initial solution for t-SNE.
        """

        if initialization_method == "PCA":
            print("Initializing with PCA...")
            X_centered = X - X.mean(axis=0)
            U, S, V = np.linalg.svd(X_centered)
            return X_centered @ V.T[:, :self.n_components]
        else:
            print(f"Initialization method {initialization_method}, is not PCA, so random initialization is used.")
            return np.random.normal(loc=0, scale=1e-4, size=(X.shape[0], self.n_components))
        
    def fit(self, X):
        """
        train t-sne for the input data.
            :param X: Training data
        returns: list of low-dimensional embeddings and the history of embeddings at each iteration.
        """
        n = X.shape[0]
    
        # Get original affinities matrix
        p_ij = self.get_original_pairwise_affinities(X, self.perplexity)
        p_ij = self.get_symmetric_p_ij(p_ij)
    
        # Initialization
        Y = np.zeros(shape=(self.iterations, n, self.n_components))
        Y[0] = np.zeros(shape=(n, self.n_components))
        Y[1] = np.array(self.initialization(X, self.initialization_method))
        costs, distances = [], []
    
        print("Optimizing Low Dimensional Embedding....")
        for t in range(1, self.iterations - 1):
                
            alpha, early_exaggeration = (0.5, self.early_exaggeration) if t < 250 else (0.8, 1)
    
            # Get Low Dimensional Affinities
            q_ij = self.get_low_dimensional_affinities(Y[t])
    
            # Get Gradient of Cost Function
            gradient = self._gradient(early_exaggeration * p_ij, q_ij, Y[t])
            Y[t + 1] = Y[t] - self.learning_rate * gradient + alpha * (Y[t] - Y[t - 1])  
    
            cost = np.sum(p_ij * np.log(p_ij / q_ij))
            costs.append(cost)    
            # Compute current value of cost function
            if t % 50 == 0 or t == 1:
                print(f"Iteration {t}: Value of Cost Function is {cost}")
    
        print(f"Completed Low Dimensional Embedding: Final Value of Cost Function is {costs[-1]}")
        embedding = Y[-1]
        return embedding, Y
    
    def plot_embedding(self, embedding, y):
        fig, ax = plt.subplots()
        g1 = ax.scatter(embedding[:, 0], embedding[:, 1], c=y, cmap="tab10")
        ax.axis("off")
        ax.set_title("MNIST t-SNE")
        plt.colorbar(g1, ax=ax)
        plt.grid(True)
        plt.show()


X = (X - X.mean(axis=0)) / X.std(axis=0)
n_sample = 1000
X_sample = X[:n_sample:, ]
y_sample = y[:n_sample]
tsne = TSNE(perplexity=10, iterations=500, learning_rate=200, early_exaggeration=4, n_components=2)
solution, Y = tsne.fit(X_sample)
tsne.plot_embedding(solution, y_sample)

# solution, Y = tsne(X_sample, perplexity=10, T=1000, η=200, early_exaggeration=4, n_dimensions=2)


Starting the high dimension computation...


  0%|          | 0/1000 [00:05<?, ?it/s]


Completed the high dimension computation...
Initialization method random, is not PCA, so random initialization is used.
Optimizing Low Dimensional Embedding....
Iteration 1: Value of Cost Function is 4.323102073012128
Iteration 50: Value of Cost Function is 2.3951779995637823
Iteration 100: Value of Cost Function is 2.1327368993575972
Iteration 150: Value of Cost Function is 1.9771285550660131
Iteration 200: Value of Cost Function is 1.9470797949593066
Iteration 250: Value of Cost Function is 1.9372367808155384
Iteration 300: Value of Cost Function is 1.1287319527167154


# 2.2(1.5pts) Using the t-SNE method, please reduce the 784 dimensions to 2 dimensions. Please try at least 5 different hyperparameters conditions.

- For each hyperparameter condition, please calculate its corresponding D and J.
- Please calculate the sum of the distance D among the 10 centroids. 
- Each centroid corresponds to each label. Since there are 10 centroids, you should calculate distance for 45 pairs. 
- Also, please calculate the objective function J.
- Your goal is to maximize the distance D among the centroids and to minimize the objective function J by optimizing the hyperparameters.

$$

D = \sum_{i,j} \|y_i - y_j\|^2

\\

J = \sum_{i=1}^{k} \sum_{j=1}^{n} \|x_i - c_j\|^2

$$

# 2.3(1.5pts)

- Please draw 2D plot for the 5 hyperparameter conditions.
- Please legend different colors for the 10 labels in the graph.